In [1]:
ls

adding from pickle.ipynb  db.sqlite3     NYT_scrape.ipynb  Untitled1.ipynb
automateDBentry.ipynb     manage.py*     static/           Untitled2.ipynb
critic/                   manatwee/      templates/        Untitled.ipynb
database_to_csv.ipynb     node_modules/  twttr/            utils/


In [2]:
cd ../

/home/ubuntu/cv/aerial/DeepNetsEO/DeepNetsForEO/nlp/site


In [3]:
import numpy as np
import scipy as sc
import pandas as pd
import csv
import matplotlib.pyplot as plt 
import seaborn as sns
import pickle
import tweepy
import re
from textblob import TextBlob
from dateutil import parser
import seaborn as sns
import time
from datetime import timedelta
from datetime import datetime

In [4]:
file = open('/home/ubuntu/cv/aerial/DeepNetsEO/DeepNetsForEO/nlp/site/src/utils/picklejar/last_run.pickle','rb')
last_run_date = pickle.load(file)

class HBO_Twitter_Sentiment(object):
    #The format of hashtagdict shall be as such:
    # { 'deuce' : [ * a list of all the hashtags to associate with the deuce * ]     }
    # since_date is YYYY-MM-SS , <string>
    # base_folder is e.g. hbo, nflx, huli, or amzn
    
    def __init__(self,mydict,base_folder,since_date):
        #   Twitter API Setup 
        consumer_key = "ZNb0uYLpAN4ADRtrTmY2smhjN"
        consumer_secret = "RtCdA6U9UKASRL8qKWjANlGdownRbtAAWWJf9eAeunLKxcfkRD"
        access_token = "17603883-PI4tf2uJ77IYKNv3Z7eSVJIgskOhj68rhYdL3AfnQ"
        access_token_secret = "JZ6ubUNrT9QN3nxiu8JgjcWk2zTA5aw0qIFx8BioaV7E3"

        auth = tweepy.OAuthHandler(consumer_key,consumer_secret)
        auth.set_access_token(access_token,access_token_secret)
        self.api = tweepy.API(auth)
        self.hashtagdict = mydict
        print self.hashtagdict
        self.base_folder = base_folder
        self.since_date = since_date
        
    
    def listToQuery(self,hashtag):
        if len(hashtag) == 1:
            query = hashtag[0]
        if len(hashtag)>1:
            query = ""
            for i,item in enumerate(hashtag):
                if i == len(hashtag)-1:
                    query = query+item
                else:    
                    query = query + item + " OR "
        return query
    
    
    def clean_tweet(self,tweet):

        #Utility function to clean tweet text by removing links, special characters
        #using simple regex statements.
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

    def get_tweet_sentiment(self, tweet):
        analysis = TextBlob(self.clean_tweet(tweet))
        if analysis.sentiment.polarity > 0:
            return 'Positive'
        elif analysis.sentiment.polarity == 0:
            return 'Neutral'
        else:
            return 'Negative'
        
    def fetch_tweet_sentiments(self):
        for key in self.hashtagdict:
            time.sleep(90)
            filename = key+'_sentiment.csv'
            path = '/home/ubuntu/cv/aerial/DeepNetsEO/DeepNetsForEO/nlp/'+self.base_folder +'/'+filename
            print path
            query = self.listToQuery(self.hashtagdict[key])
            
            # Open/Create a file to append data
            csvFile = open(path, 'a')
            #Use csv Writer
            csvWriter = csv.writer(csvFile)
            api_query = tweepy.Cursor(self.api.search,q=query,count=500,lang="en",since=self.since_date).items()
            for tweet in api_query:
                print (tweet.created_at, tweet.author._json['screen_name'], tweet.text)
                csvWriter.writerow([tweet.created_at,tweet.author._json['screen_name'],tweet.text.encode('utf-8')])
                
    def generateNetPosDataFrame(self):
        for key in self.hashtagdict:
            filename = key+'_sentiment.csv'
            path = '/home/ubuntu/cv/aerial/DeepNetsEO/DeepNetsForEO/nlp/'+self.base_folder +'/'+filename
            show_df = pd.read_csv(path, names=['datetime','tweet'])
            show_df.dropna(inplace=True)
            show_df['sentiment'] = show_df.tweet.apply(self.get_tweet_sentiment)
            show_df.datetime = show_df.datetime.apply(parser.parse)
            show_df.set_index(show_df.datetime,inplace=True)
            show_grouped = show_df.groupby([pd.TimeGrouper(freq='D'),'sentiment',]).size()
            show_pct = show_grouped.groupby(level=0).apply(lambda x: 100 * x/float(x.sum()))
            show_NNP = show_pct.swaplevel(i=-1,j=-2).sort_values().unstack().transpose()
            show_NNP['NetPositive'] = show_NNP['Positive'] - show_NNP['Negative']
            show_grouped=show_grouped.swaplevel(i=-1,j=-2).sort_values().unstack().transpose()
            show_grouped['TotalTweets'] = show_grouped['Positive'] + show_grouped['Negative'] + show_grouped['Neutral']
            show_NNP['TotalTweets'] = show_grouped['TotalTweets']
            return show_NNP, show_df
        
    def generatePlots(self):
        for key in self.hashtagdict:
            print key
            filename = key+'_sentiment.csv'
            path = '/home/ubuntu/cv/aerial/DeepNetsEO/DeepNetsForEO/nlp/'+self.base_folder +'/'+filename
            show_df = pd.read_csv(path, names=['datetime','username','tweet'])
            show_df.dropna(inplace=True)
            show_df.drop_duplicates(inplace=True)
            show_df['sentiment'] = show_df.tweet.apply(self.get_tweet_sentiment)
            show_df.datetime = show_df.datetime.apply(parser.parse)
            show_df.set_index(show_df.datetime,inplace=True)
            show_grouped = show_df.groupby([pd.TimeGrouper(freq='D'),'sentiment',]).size()
            show_pct = show_grouped.groupby(level=0).apply(lambda x: 100 * x/float(x.sum()))
            show_NNP = show_pct.swaplevel(i=-1,j=-2).sort_values().unstack().transpose()
            if show_NNP.shape[1] <3:
                continue
            if 'Negative' in show_NNP.columns and 'Positive' in show_NNP.columns:
                show_NNP['NetPositive'] = show_NNP['Positive'] - show_NNP['Negative']
            elif 'Negative' in show_NNP.columns:
                show_NNP['NetPositive'] = -show_NNP['Negative']
            elif 'Positive' in show_NNP.columns:
                show_NNP['NetPositive'] = show_NNP['Positive']
            show_grouped=show_grouped.swaplevel(i=-1,j=-2).sort_values().unstack().transpose()
            if 'Negative' in show_NNP.columns and 'Positive' in show_NNP.columns and 'Neutral' in show_NNP.columns:
                show_grouped['TotalTweets'] = show_grouped['Positive'] + show_grouped['Negative'] + show_grouped['Neutral']
            elif 'Negative' in show_NNP.columns and 'Neutral' in show_NNP.columns:
                show_grouped['TotalTweets'] = show_grouped['Negative'] + show_grouped['Neutral']
            elif 'Positive' in show_NNP.columns and 'Neutral' in show_NNP.columns:
                show_grouped['TotalTweets'] = show_grouped['Positive'] + show_grouped['Neutral']
            elif 'Positive' in show_NNP.columns and 'Negative' in show_NNP.columns:
                show_grouped['TotalTweets'] = show_grouped['Positive'] + show_grouped['Negative']
            elif 'Negative' in show_NNP.columns:
                show_grouped['TotalTweets'] = show_grouped['Negative']
            elif 'Positive' in show_NNP.columns:
                show_grouped['TotalTweets'] = show_grouped['Positive']
            elif 'Neutral' in show_NNP.columns:
                show_grouped['TotalTweets'] = show_grouped['Neutral']
            show_NNP['TotalTweets'] = show_grouped['TotalTweets']
            fig_show, ax1_show = plt.subplots()
            ax2_show = ax1_show.twinx()
            if 'NetPositive' in show_NNP.columns:
                ax1_show.plot(show_NNP.NetPositive,'g',linewidth=5)
            ax2_show.bar(show_NNP.index,show_NNP.TotalTweets,align='center',alpha=0.5)
            ax1_show.set_ylim([-100,100])
            ax1_show.set_xlabel('Date')
            ax1_show.set_ylabel('Net Positive Sentiment %')
            ax2_show.set_ylabel('Total Tweets')
            fig_show.autofmt_xdate()
            plt.title(key)
            #fig_show.show()
    
    def tweetDataFrames(self):
        dataframedict = {}
        for key in self.hashtagdict:
            filename = key+'_sentiment.csv'
            path = '/home/ubuntu/cv/aerial/DeepNetsEO/DeepNetsForEO/nlp/' + self.base_folder+ '/'+filename
            show_df = pd.read_csv(path, names=['datetime','username','tweet'])
            show_df.dropna(inplace=True)
            show_df.drop_duplicates(inplace=True)
            show_df['sentiment'] = show_df.tweet.apply(self.get_tweet_sentiment)
            show_df.datetime = show_df.datetime.apply(parser.parse)
            show_df.set_index(show_df.datetime,inplace=True)
            dataframedict[key] = show_df
        for k in dataframedict.keys():
            dataframedict[k]['show'] = k
            dataframedict[k]['ntwk'] = self.base_folder
        df = pd.concat([dataframedict[x] for x in dataframedict.keys()])
        return df

In [12]:
got = {'game_of_thrones':['#GameOfThrones']}
file = open('/home/ubuntu/cv/aerial/DeepNetsEO/DeepNetsForEO/nlp/site/src/utils/picklejar/last_run.pickle','rb')
last_run_date = pickle.load(file)
dt = last_run_date.strftime('%Y-%m-%d')

GOT = HBO_Twitter_Sentiment(got,'hbo',dt)


{'game_of_thrones': ['#GameOfThrones']}


In [13]:
GOT_dfs = GOT.tweetDataFrames()

ValueError: month must be in 1..12

In [54]:
key = 'game_of_thrones'
base_folder = 'hbo'
filename = key+'_sentiment.csv'
path = '/home/ubuntu/cv/aerial/DeepNetsEO/DeepNetsForEO/nlp/' + base_folder+ '/'+filename
show_df = pd.read_csv(path, names=['datetime','username','tweet'])

In [55]:
show_df

,datetime,username,tweet
0,11/11/2017 2:00,MiloInMotion,RT @GOTtheMusical: We're inviting @Lesdoggg to...
1,11/11/2017 2:00,ReadLOTR,RT @GOTtheMusical: We're inviting @Lesdoggg to...
2,11/11/2017 2:00,BarryPi,"If you ever call me sister again, Ill have you..."
3,11/11/2017 2:00,TriviaAD,#GameOfThrones #Trivia @Abbeytavern3ave Tuesda...
4,11/11/2017 1:57,jeka_mota,RT @kitharingtoncom: How completely adorable i...
5,11/11/2017 1:56,ImperioMundial,Which is the better series of the world-wide T...
6,11/11/2017 1:55,JustineAnnaW,Bruh since #GameofThrones season 8 wont be a p...
7,11/11/2017 1:54,brisa_rusher77,RT @thronesfacts: Jason Momoa with members of ...
8,11/11/2017 1:54,treasurotica,RT @treasurotica: #GameOfThrones Coffee Mug Ho...
9,11/11/2017 1:53,thelzrdking,RT @thronesfacts: Jason Momoa with members of ...


In [56]:
show_df.dropna(inplace=True)


In [57]:
show_df.drop_duplicates(inplace=True)


In [58]:
def get_tweet_sentiment(tweet):
    analysis = TextBlob(clean_tweet(tweet))
    if analysis.sentiment.polarity > 0:
        return 'Positive'
    elif analysis.sentiment.polarity == 0:
        return 'Neutral'
    else:
        return 'Negative'

In [59]:
def clean_tweet(tweet):

    #Utility function to clean tweet text by removing links, special characters
    #using simple regex statements.
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())


In [60]:
show_df['sentiment'] = show_df.tweet.apply(get_tweet_sentiment)


In [61]:
show_df.datetime = show_df.datetime.apply(parser.parse)

In [53]:
show_df[71930:71940].datetime

132328      2017-12-07 18:32:45
132329      2017-12-07 18:32:43
132330      2017-12-07 18:32:33
132331      2017-12-07 18:32:26
132332      2017-12-07 18:32:15
132333    202017-12-17 22:34:23
132334      2017-12-17 22:33:42
132335      2017-12-17 22:32:55
132336      2017-12-17 22:32:54
132337      2017-12-17 22:32:00
Name: datetime, dtype: object

README.md  src/


In [62]:
picklepath = 'utils/picklejar/TV_new.pickle'

In [68]:
tv = pickle.load(open(picklepath))

In [67]:
cd site/src/

/home/ubuntu/cv/aerial/DeepNetsEO/DeepNetsForEO/nlp/site/src


In [71]:
tv[tv.show == 'dunkirk']

,datetime,ntwk,sentiment,show,tweet,username
datetime,,,,,,
2017-12-18 02:26:46,2017-12-18 02:26:46,movies,Positive,dunkirk,RT @dunkirkmovie: #Dunkirk has been nominated ...,Georgeolwande
2017-12-18 02:25:13,2017-12-18 02:25:13,movies,Positive,dunkirk,RT @iTunes: Experience this astonishing chapte...,DebieYusran
2017-12-18 02:10:25,2017-12-18 02:10:25,movies,Neutral,dunkirk,"RT @CatchNews: #Baahubali2 surpasses #Dunkirk,...",ivk_45
2017-12-18 02:01:35,2017-12-18 02:01:35,movies,Neutral,dunkirk,"RT @CatchNews: #Baahubali2 surpasses #Dunkirk,...",ChoudharyTeeja
2017-12-18 01:57:25,2017-12-18 01:57:25,movies,Neutral,dunkirk,RT @dunkirkmovie: What was your experience wat...,zhouchen19991
2017-12-18 01:44:39,2017-12-18 01:44:39,movies,Neutral,dunkirk,Yeahhh... #Dunkirk was kinda whack.,_noeeezy
2017-12-18 01:43:52,2017-12-18 01:43:52,movies,Neutral,dunkirk,"RT @CatchNews: #Baahubali2 surpasses #Dunkirk,...",iam_RebelFann
2017-12-18 01:31:03,2017-12-18 01:31:03,movies,Neutral,dunkirk,"RT @CatchNews: #Baahubali2 surpasses #Dunkirk,...",pavansaaho21
2017-12-18 01:29:18,2017-12-18 01:29:18,movies,Positive,dunkirk,Finally found some time. And I watched Dunkirk...,vincenzzo


In [74]:
datefile = 'utils/picklejar/last_run.pickle'

In [75]:
dt=pickle.load(open(datefile)).strftime('%Y-%m-%d')

In [76]:
dt

'2017-12-18'

In [77]:
addTV=tv[tv.datetime>=dt]

In [78]:
addTV

,datetime,ntwk,sentiment,show,tweet,username
datetime,,,,,,
2017-12-18 00:21:55,2017-12-18 00:21:55,hbo,Neutral,silicon_valley,Still cracking up that the 3 commas guy is the...,elisal
2017-12-18 00:07:36,2017-12-18 00:07:36,hbo,Positive,silicon_valley,RT @SiliconHBO: We're inviting one lucky fan t...,mytweetmark
2017-12-18 00:05:43,2017-12-18 00:05:43,hbo,Neutral,silicon_valley,RT @SiliconHBO: Thomas Middleditch has an offe...,mytweetmark
2017-12-18 00:36:17,2017-12-18 00:36:17,hbo,Negative,westworld,I just watched the first episode of #Westworld...,Tambulla
2017-12-18 00:32:41,2017-12-18 00:32:41,hbo,Positive,westworld,RT @IndieWire: Watch: #Westworld Season 2 and ...,alvaro21499
2017-12-18 00:31:36,2017-12-18 00:31:36,hbo,Positive,westworld,"RT @_secretartist_: For #meals4mouse,here's mo...",jsimpsonfansite
2017-12-18 00:26:28,2017-12-18 00:26:28,hbo,Positive,westworld,RT @IndieWire: Watch: #Westworld Season 2 and ...,kingmostaza
2017-12-18 00:26:18,2017-12-18 00:26:18,hbo,Positive,westworld,RT @IndieWire: Watch: #Westworld Season 2 and ...,kdefreita94
2017-12-18 00:20:32,2017-12-18 00:20:32,hbo,Neutral,westworld,I just finished #westworld https://t.co/Qkd1Zx...,semperdux
